# SETUP

## imports

In [103]:
import string
import numpy as np
from itertools import cycle
import requests
import collections
from pprint import pprint
import operator
import networkx as nx


## constants

In [104]:
lowercase = string.ascii_lowercase
uppercase = string.ascii_uppercase

## helpers

In [105]:
def get_level_input(lvl_num):
    with open('advent_inputs/%d.txt' %lvl_num) as f:
        level_input=f.read()
        return level_input[:-1]
    
def print_result(answer):
    pprint("RESULT: "+str(answer))
    print()
    pprint("TIME"+"."*60)
    
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# LEVEL 1

## setup

In [6]:
frequencies=get_level_input(1)
frequencies=frequencies.splitlines()
frequencies=[int(x[1:]) if x[0]=='+' else int(x) for x in frequencies]

## part one

In [7]:
%%time
print_result(sum(frequencies))

'RESULT: 500'

'TIME............................................................'
CPU times: user 150 µs, sys: 88 µs, total: 238 µs
Wall time: 180 µs


## part two

In [8]:
%%time
y=set({})
current_sum = 0
for i in cycle(frequencies):
    current_sum+=i
    if current_sum in y:
        print_result(current_sum)
        break
    else:
        y.add(current_sum)

'RESULT: 709'

'TIME............................................................'
CPU times: user 43.7 ms, sys: 5.18 ms, total: 48.9 ms
Wall time: 47.3 ms


# LEVEL 2

## setup

In [9]:
input_boxes=get_level_input(2)
input_boxes=input_boxes.split('\n')
count_boxes = [collections.Counter(a) for a in input_boxes]

## part one

In [10]:
%%time
twos=0
threes=0
for box_id in count_boxes:
    if 2 in box_id.values():
        twos+=1
    if 3 in box_id.values():
        threes+=1
print_result(str(twos*threes))

'RESULT: 6422'

'TIME............................................................'
CPU times: user 3.28 ms, sys: 113 µs, total: 3.39 ms
Wall time: 3.3 ms


## part two

In [11]:
%%time
for i, box_one in enumerate(input_boxes):
    for box_two in input_boxes[i+1:]:
        diff=0
        wrong_letter_index=0
        for j in range(len(box_one)):
            if box_one[j]==box_two[j]: continue
            diff+=1
            wrong_letter_index=j
            if(diff>1): break
        if(diff==1): 
            print_result(box_one[:wrong_letter_index]+box_one[wrong_letter_index+1:])
            raise StopExecution     

'RESULT: qcslyvphgkrmdawljuefotxbh'

'TIME............................................................'


# LEVEL 3

## setup

In [12]:
fabric_list=get_level_input(3).splitlines()
fabric_list=[x.split(" ") for x in fabric_list]

## part one

In [13]:
%%time
fabric=np.zeros((1000,1000))
for i in fabric_list:
    dim=tuple([int(z) for z in i[2][:-1].split(",")])
    size=tuple([int(z) for z in i[3].split("x")])
    fabric[dim[0]:dim[0]+size[0],dim[1]:dim[1]+size[1]]+=1
    NUM_SINGLE_USED=(fabric > 1).sum()
print_result(NUM_SINGLE_USED)

'RESULT: 101565'

'TIME............................................................'
CPU times: user 1.81 s, sys: 9.44 ms, total: 1.82 s
Wall time: 1.82 s


## part two

In [14]:
%%time
for i in fabric_list:
    dim=tuple([int(z) for z in i[2][:-1].split(",")])
    size=tuple([int(z) for z in i[3].split("x")])
    if((fabric[dim[0]:dim[0]+size[0],dim[1]:dim[1]+size[1]]==1).all()):
        print_result(i[0])

'RESULT: #656'

'TIME............................................................'
CPU times: user 19.6 ms, sys: 3.32 ms, total: 23 ms
Wall time: 20.8 ms


# LEVEL 4

## setup

In [15]:
guards = get_level_input(4)
guards = guards.splitlines()
guards.sort()
guards = [[x[0][-2:], x[1][1:]] for x in [j.split(']') for j in guards]]

## part one

In [16]:
%%time

cur_guard=0
guard_to_min={}
for i, log in enumerate(guards):
    if(log[1][0]=='G'):
        cur_guard=int(log[1].split(" ")[1][1:])
        if cur_guard not in guard_to_min.keys():
            guard_to_min[cur_guard]=np.zeros(60)
    elif(log[1][0]=='f'):
        if(int(log[0])<int(guards[i+1][0])):
            guard_to_min[cur_guard][int(log[0]):int(guards[i+1][0])]+=1
        else:
            guard_to_min[cur_guard][int(log[0]):]+=1
            guard_to_min[cur_guard][:int(guards[i+1][0])]+=1
m = max([sum(v) for v in guard_to_min.values()])

for key,value in guard_to_min.items():
    if( m==sum(value)):
        print_result(list(value).index(max(value))*key)

'RESULT: 102688'

'TIME............................................................'
CPU times: user 3.51 ms, sys: 342 µs, total: 3.85 ms
Wall time: 3.56 ms


## part two

In [17]:
%%time

cur_guard=0
guard_to_min={}
for i, log in enumerate(guards):
    if(log[1][0]=='G'):
        cur_guard=int(log[1].split(" ")[1][1:])
        if cur_guard not in guard_to_min.keys():
            guard_to_min[cur_guard]=np.zeros(60)
    elif(log[1][0]=='f'):
        if(int(log[0])<int(guards[i+1][0])):
            guard_to_min[cur_guard][int(log[0]):int(guards[i+1][0])]+=1
        else:
            guard_to_min[cur_guard][int(log[0]):]+=1
            guard_to_min[cur_guard][:int(guards[i+1][0])]+=1
m = max(i for v in guard_to_min.values() for i in v)

for key,value in guard_to_min.items():
    if( m in value ):
        print_result(list(value).index(m)*key)

'RESULT: 56901'

'TIME............................................................'
CPU times: user 3.46 ms, sys: 190 µs, total: 3.65 ms
Wall time: 3.6 ms


# LEVEL 5

## setup

In [18]:
polymers=get_level_input(5)

## part one

In [19]:
%%time
polymer_temp=polymers
i=0
while i!=len(polymer_temp)-1:
    if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
        polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
        i=max(0,i-1)
    else:
        i+=1
print(len(polymer_temp))

9202
CPU times: user 95.2 ms, sys: 5.14 ms, total: 100 ms
Wall time: 99 ms


## part two

In [20]:
%%time
scores=[]
for letter in lowercase:
    polymer_temp=polymers
    i=0
    polymer_temp=polymer_temp.translate(str.maketrans('', '', letter+letter.upper()))
    while i!=len(polymer_temp)-1:
        if(abs(ord(polymer_temp[i])-ord(polymer_temp[i+1]))==32):
            polymer_temp=polymer_temp[0:i]+polymer_temp[i+2:]
            i=max(0,i-1)
        else:
            i+=1
    scores.append(len(polymer_temp))
print_result(min(scores))

'RESULT: 6394'

'TIME............................................................'
CPU times: user 2.21 s, sys: 70.5 ms, total: 2.28 s
Wall time: 2.28 s


# LEVEL 6

## setup

In [24]:
GRID_SIZE=1000
SHIFT_SIZE=50

coords=get_level_input(6)
coords_list=[tuple([int(coord.split(", ")[0]), int(coord.split(", ")[1])]) for coord in coords.splitlines()]
shift_coords_list = [(x[0]+SHIFT_SIZE, x[1]+SHIFT_SIZE) for x in coords_list]

def l1(c1, c2):
    return abs(c1[0]-c2[0])+abs(c1[1]-c2[1])

def get_index_min_l1(pos):
    coord_min=[]
    for coord in shift_coords_list:
        coord_min.append(l1(coord, pos))
    x=np.where(np.array(coord_min) == np.array(coord_min).min())
    if(len(x[0])>1): return 0
    return x[0][0]

def determine_inf_values():
    inf_set=set({})
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if 0 in {i, j} or GRID_SIZE-1 in {i,j}:
                inf_set.add(get_index_min_l1((i,j))+1)
    return inf_set

inf_set=determine_inf_values()

In [129]:
# TEST_COORDS
# GRID_SIZE=20
# SHIFT_SIZE=5
# coords_list=[(1, 1),(1, 6),(8, 3),(3, 4),(5, 5),(8, 9)]
# shift_coords_list = [(x[0]+SHIFT_SIZE, x[1]+SHIFT_SIZE) for x in shift_coords_list]
# coords= set(coord_list)

# print(coords_list)

## part one

### thoughts

- Need to determine which areas are infinite
    - are these automatically the points that have the lowest or highest x or y coordinate?
- Need to limit scope and calculate each grdp points relation to our coordinates
- Assign the minimum distance to a points area score
- Pick the point that has the minimum area score that is not infiinite

### EPIPHANY
CHANGING THE SIZE OF THE GRID WILL NOT CHANGE THE NUMBER OF RESULTS FOR NON-INFINITE OPTIONS. THIS CAN ONLY BE THE CASE FOR NON INF OPTIONS SO TRY TWO GRIDS ONE OF SMALL SIZE, ONF OF LARGER SIZE AND CHOOSE MAX VALUE THAT DOESN"T CHANGE.

**NOTE**: the grid size must expand in all directions for this method to work. This means points need to be recentered

**NOTTE**: easier solutiton, any point that owns one of thte outer rings will be infinite(given a significant enough buffer around all points


In [25]:
%%time

test_grid = np.zeros((GRID_SIZE,GRID_SIZE))
for i, coord in enumerate(shift_coords_list):
    test_grid[coord[0], coord[1]]=i+1
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if 0 not in {i, j} or GRID_SIZE-1 not in {i,j}:
            min_value=get_index_min_l1((i,j))+1
            if(min_value not in inf_set):
                test_grid[i][j]=min_value
largest_non_inf_area = int(collections.Counter(list(test_grid.flatten())).most_common(2)[1][1])
print_result(largest_non_inf_area)


'RESULT: 3223'

'TIME............................................................'
CPU times: user 37.9 s, sys: 1.02 s, total: 38.9 s
Wall time: 38.3 s


## part two

In [30]:
%%time

THRESHOLD_DISTANCE=10000
SAFE_POINTS=[]
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if(sum([l1((i,j),x) for x in shift_coords_list])<THRESHOLD_DISTANCE):
            SAFE_POINTS.append((i,j))
print_result(len(SAFE_POINTS))


'RESULT: 40495'

'TIME............................................................'
CPU times: user 21.7 s, sys: 47.5 ms, total: 21.8 s
Wall time: 21.9 s


# LEVEL 7

## setup

In [111]:
steps = get_level_input(7).splitlines()
print(len(steps))
num_steps=26
uppercase = string.ascii_uppercase

def to_node(step):
    return(step[1:-1].translate(str.maketrans('','',lowercase)).split())

101


In [108]:
# temp_test
steps=[
"Step C must be finished before step A can begin.",
"Step C must be finished before step F can begin.",
"Step A must be finished before step B can begin.",
"Step A must be finished before step D can begin.",
"Step B must be finished before step E can begin.",
"Step D must be finished before step E can begin.",
"Step F must be finished before step E can begin."
]

num_steps=6

uppercase="ABCDEF"

def to_node(step):
    return(step[1:-1].translate(str.maketrans('','',lowercase)).split())

##  part one

In [127]:
locks={}
_steps=map(to_node, steps)
solution=[]

for step in _steps:
    locks.setdefault(step[1],[]).append(step[0])

start_nodes = sorted(list(set(list(uppercase))-set(locks.keys())))
start_node, remaining_nodes = start_nodes[0], start_nodes[1:]
solution+=start_node
while(len(solution)<num_steps):
    next_step=list(set(remaining_nodes)-set(solution))
    for key, dependencies in locks.items():
        if (set(dependencies) <= set(solution)) and key not in solution:
            next_step.append(key)
    print(sorted(next_step[0]))
    solution+=(sorted(next_step)[0])
    print(solution)
print(len(solution))
print_result("".join(solution))

['F']
['C', 'F']
['G']
['C', 'F', 'G']
['M']
['C', 'F', 'G', 'H']
['M']
['C', 'F', 'G', 'H', 'A']
['M']
['C', 'F', 'G', 'H', 'A', 'E']
['M']
['C', 'F', 'G', 'H', 'A', 'E', 'M']
['N']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N']
['B']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B']
['P']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P']
['R']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R']
['D']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R', 'D']
['S']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R', 'D', 'I']
['S']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R', 'D', 'I', 'S']
['V']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R', 'D', 'I', 'S', 'V']
['W']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R', 'D', 'I', 'S', 'V', 'W']
['Q']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R', 'D', 'I', 'S', 'V', 'W', 'Q']
['U']
['C', 'F', 'G', 'H', 'A', 'E', 'M', 'N', 'B', 'P', 'R', 'D', 'I', 'S', 'V', 'W', 'Q', 'U']
['Z']
['C', 'F', 'G', 'H', 'A',